In [2]:
import os

import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from keras_preprocessing.image import ImageDataGenerator

In [4]:
def read_file(patient_id, contrast):
    filenames = [file_pattern for tumour in tumours]
    try:
        x = nib.load(filenames[0])
    except FileNotFoundError:
        x = nib.load(filenames[1])
    filenames.replace()
    nii_data = x.get_fdata()
    return nii_data

In [3]:
tumours = ['LGG','HGG']
prefix = 'C:/Users/Fungj/Documents/EECE_571F/MICCAI_BraTS_2018_Data_Training'
training_set = f'{prefix}/MICCAI_BraTS_2018_Data_Training/survival_data.csv'
file_pattern = '{prefix}/MICCAI_BraTS_2018_Data_Training/{tumour}/{patient_id}/{patient_id}_{contrast}.nii.gz'
patient_id = 'Brats18_TCIA09_620_1'
contrasts = ['t1ce', 'flair', 't1', 't2']
tumours = ['LGG', 'HGG']


In [4]:
metadata = pd.read_csv(training_set)

In [5]:
def create_filename(row, contrast):
    possible_filenames = [file_pattern.format(prefix=prefix, tumour=tumour, patient_id=row['BraTS18ID'], contrast=contrast) for tumour in tumours]
    filename = [filename for filename in possible_filenames if os.path.exists(filename)][0]
    return filename

In [8]:
metadata['filename_flair'] = metadata.apply(create_filename, axis=1, contrast='flair')
metadata['filename_t1ce'] = metadata.apply(create_filename, axis=1, contrast='t1ce')
metadata['filename_seg'] = metadata.apply(create_filename, axis=1, contrast='seg')

In [9]:
metadata

,BraTS18ID,Age,Survival,ResectionStatus,filename_flair,filename_t1ce,filename_seg
0,Brats18_TCIA08_167_1,74.907,153,NaN,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...
1,Brats18_TCIA08_242_1,66.479,147,NaN,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...
2,Brats18_TCIA08_319_1,64.860,254,NaN,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...
3,Brats18_TCIA08_469_1,63.899,519,NaN,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...
4,Brats18_TCIA08_218_1,57.345,346,NaN,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...
...,...,...,...,...,...,...,...
158,Brats18_CBICA_ABB_1,68.493,465,GTR,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...
159,Brats18_CBICA_AAP_1,39.068,788,GTR,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...
160,Brats18_CBICA_AAL_1,54.301,464,GTR,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...
161,Brats18_CBICA_AAG_1,52.263,616,GTR,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...,/home/atom/Documents/datasets/brats/MICCAI_Bra...


# Jason's Implementation

## Load Data in Batches

In [ ]:
# edit this function tomorrow
def load_img(img_dir, img_list):
    images=[]
    for i, image_name in enumerate(img_list):    
        if (image_name.split('.')[1] == 'npy'):
            
            image = np.load(img_dir+image_name)
                      
            images.append(image)
    images = np.array(images)
    
    return(images)

# edit this function tomorrow
def imageLoader(img_dir, img_list, mask_dir, mask_list, batch_size):

    L = len(img_list)

    #keras needs the generator infinite, so we will use while true  
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)
                       
            X = load_img(img_dir, img_list[batch_start:limit])
            Y = load_img(mask_dir, mask_list[batch_start:limit])

            yield (X,Y) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size   
            batch_end += batch_size

In [46]:
tumour = 'HGG'
prefix = 'C:/Users/Fungj/Documents/EECE_571F/MICCAI_BraTS_2018_Data_Training/'
file_dir = prefix + tumour
dataset_file_list = os.listdir(file_dir)
contrasts = os.listdir(os.path.join(file_dir,dataset_file_list[2]))

batch_size = 32

# shuffle and split the dataset file list
import random
random.seed(42)
file_list_shuffled = dataset_file_list.copy()
random.shuffle(file_list_shuffled)
test_ratio = 0.2

train_file, test_file = file_list_shuffled[0:int(len(file_list_shuffled)*(1-test_ratio))], file_list_shuffled[int(len(file_list_shuffled)*(1-test_ratio)):]






In [47]:
train_file[167]

'Brats18_CBICA_AVV_1'

In [48]:
test_file[41]

'Brats18_TCIA02_608_1'

In [42]:
file_list_shuffled[-1]

'Brats18_TCIA02_608_1'